In [4]:
from db import call_db, call_ignition, call_db_json, update_ignition, update_db, force_cancel_rail, config, update_ignition_with_values
from ml_model import upcomming_demand_prediction, predict_upcoming_hk
from queries import update_processing, proc
from util_func import ProcessingStorage, DemandStorage, StockStorage, DiecastStorage, get_mondays, Kanban
import pandas as pd
import numpy as np
import difflib
#!/usr/bin/python
import psycopg2
from configparser import ConfigParser
from psycopg2.extras import RealDictCursor
from datetime import timedelta, date, datetime
import math
from db import insert_many_with_df 
import os
import re

throttle_2880_2841.pth


In [12]:
class HkUnreleased:
    def __init__(self):
        self.empty = True

    def set_data(self, db_response, due):
        self.empty = False
        self.all_df = pd.DataFrame.from_dict(db_response)

        grouped_df = self.all_df.groupby(["item_number", due]).first()
        grouped_df["order_quantity"] = self.all_df.groupby(["item_number", due])[
            "order_quantity"
        ].sum()
        grouped_df["mos"] = self.all_df.groupby(["item_number", due]).agg(
            {"order_number": " ".join}
        )
        grouped_df = grouped_df.reset_index()
        self.predicted_df = predict_upcoming_hk(grouped_df, due)
        sorted_by_due = self.all_df.sort_values([due, 'order_number'])

        grouped_df_for_release = sorted_by_due.groupby("item_number").first()
        grouped_df_for_release["order_quantity"] = sorted_by_due.groupby("item_number")[
            "order_quantity"
        ].sum()
        grouped_df_for_release["mos"] = sorted_by_due.groupby(["item_number"]).agg(
            {"order_number": " ".join}
        )        
        self.temp_data = sorted_by_due.groupby(["item_number"])['order_number']       
        
        grouped_df_for_release["hks"] = sorted_by_due.groupby(["item_number"]).agg(
            {"reference_number": " ".join}
        )
        grouped_df_for_release = grouped_df_for_release.reset_index()
        predicted_df_release = predict_upcoming_hk(grouped_df_for_release, due)
        ### for
        self.GY3 = predicted_df_release[
            (predicted_df_release["facility"].str[:5] == "C0010")
            & (predicted_df_release["product_length"] >= 160)
        ].copy()
        self.GY4 = predicted_df_release[
            (predicted_df_release["facility"].str[:5] == "D0010")
            | (
                (predicted_df_release["facility"].str[:5] == "C0010")
                & (predicted_df_release["product_length"] < 160)
            )
        ].copy()
sql = f"""
    SELECT
        order_number, product_family, product_model, item_description, printed_due, item_number, product_g1, product_pitch, reference_number,product_length,order_quantity, order_scheduled_due,
        (
        SELECT
            string_agg(facility_id, ' ')
        FROM manufacturing_order_processes
            WHERE manufacturing_order_processes.order_number = manufacturing_orders.order_number
        ) facility
    FROM manufacturing_orders
    WHERE manufacturing_orders.order_status ='10'
        AND manufacturing_orders.order_release_code = 5
        AND order_scheduled_due <= CURRENT_DATE + integer '21'
        AND product_length IS NOT NULL
        AND product_model IS NOT NULL
        AND reference_number LIKE 'HK%'
        AND position('RAIL'in item_description) > 0        
    """
hk_unreleased = HkUnreleased()

db_response = call_db_json(sql)
hk_unreleased.set_data(db_response, "order_scheduled_due")
test_df = pd.DataFrame(hk_unreleased.temp_data)

c:\Users\Ejeong\flask\ml_model.py:526: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  input["start_due"] = input[due] - input["pred"]
c:\Users\Ejeong\flask\ml_model.py:526: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized.
  input["start_due"] = input[due] - input["pred"]


In [13]:
test_df

,0,1
0,200235200912400,"50 M225367 Name: order_number, dtype: object"
1,201715175904550,52 M226220 53 M226221 54 M226222 Name...
2,301751015367500,56 M196053 57 M196054 58 M196055 Name...
3,301751018331100,42 M217632 41 M217631 40 M217630 39 ...
4,301751019579600,"51 M225377 Name: order_number, dtype: object"
5,301751023062700,18 M213675 19 M213676 Name: order_number...
6,301751023095900,12 M213516 13 M213517 Name: order_number...
7,301761000039300,"45 M221345 Name: order_number, dtype: object"
8,302451006395700,"2 M182343 Name: order_number, dtype: object"
9,302451017413600,"26 M216922 Name: order_number, dtype: object"
